In [8]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os

data = "combined_data/combined_conn.log.labeled"

# Split the dataset into 4 parts in different files to avoid memory error
def split_data(data, output_files, chunksize=10000):
    # Create writers for each file
    writers = [open(f, "w") for f in output_files]
    
    # Read data in chunks to avoid memory overload
    with pd.read_csv(data, sep='\t', comment="#", header=None, chunksize=chunksize) as reader:
        # Initialize index for distributing rows across files
        i = 0
        total_rows = sum(1 for row in open(data, 'r', encoding='utf-8') if not row.startswith('#'))
        
        # Iterate over chunks and distribute the rows to 4 files
        for chunk in tqdm(reader, total=total_rows // chunksize):
            # Extract column names from the 7th row (index 6)
            if i == 0:
                chunk.columns = chunk.iloc[6][1:].values
                chunk = chunk.drop(6).reset_index(drop=True)
            
            # Split the chunk into 4 parts and write to respective files
            for j, split_chunk in enumerate(np.array_split(chunk, 4)):
                split_chunk.to_csv(writers[j], sep='\t', index=False, header=i==0)
                
            i += 1

    # Close all writers
    for writer in writers:
        writer.close()

# Output file paths
output_files = [
    "combined_data/combined_conn.log.labeled1",
    "combined_data/combined_conn.log.labeled2",
    "combined_data/combined_conn.log.labeled3",
    "combined_data/combined_conn.log.labeled4"
]

split_data(data, output_files)
print("Data has been split into 4 parts")

  0%|          | 0/32530 [00:00<?, ?it/s]


ValueError: Length mismatch: Expected axis has 21 elements, new values have 20 elements